# Final assignment of “Management and Analysis of Physics Datasets” - Part 2

## Dr. Andreas-Joachim Peters - CERN

Author: Nguyen Xuan Tung 

ID: 2005491

## 0. Fun exercise

 1 + 2 = 3
 
 2 + 5 = 7
 
 3 + 7 = 4 
 
 4 + 5 = ?
 
 5 + 9 = 12
 

You might have seen this kind of puzzles on social media. It should be
straight-forward for you to understand which operations connect the two
numbers before the = sign. What is the result of the ? field ? Show proof
for each line!

Rewrite the digit as binary: 

 1 + 2 = 3 => f(01,10) = 11
 
 2 + 5 = 7 => f(010,101) = 111
 
 3 + 7 = 4 => f(011,111) = 100
 
 4 + 5 = ? => f(100,101) = 001
 
 5 + 9 = 12 => f(0101,1001) = 1100
 
 001 = 1 so 4 + 5 = 1



The result: 
 
 1 + 2 = 3
 
 2 + 5 = 7
 
 3 + 7 = 4 
 
 4 + 5 = 1
 
 5 + 9 = 12

## 1. Redundancy

We are programming a file based RAID-4 software algorithm. For this
purpose we are converting a single input (raid4.input) file into four data
files raid4.0,raid4.1,raid4.2,raid4.3 and one parity file
raid4.4 - the four data and one parity file we call ‘stripe files’.

The input file can be downloaded from:

http://apeters.web.cern.ch/apeters/pd2021/raid4.input

To do this we are reading in a loop sequentially blocks of four bytes from the input
file until the whole file is read:

• in each loop we write one of the four read bytes round-robin to each data file,
compute the parity of the four input bytes and write the result into the fifth parity
file. ( see the drawing for better understanding )

• we continue until all input data has been read. If the last bytes read from the input
file are not filling four bytes, we consider the missing bytes as zero for the parity
computation.

In [17]:
import numpy as np

def get_arr(path): # function get the file as an array
    with open(path,'r+b') as file:
        v = np.frombuffer(file.read(),dtype = np.uint8)
    return v


def post_arr(arr, path): # function write array to file, convert to bytes
    with open(path,'w+b') as file:
        file.write(arr.tobytes())
    return


def xor_arr(matrix):# function applies the bitwise xor function to a collection of arrays, given as rows in a matrix
    # Neutral xor array
    v = np.zeros(len(matrix[0]),dtype = np.uint8)
    # Applying bitwise_xor between the matrix rows
    for arr in matrix:
        v = np.bitwise_xor(v,arr)
    return v

### 1.1 Write a program (C,C++, R or Python), which produces four striped data and one parity file as described above using the given input file hint: if you have a problem programming this yourself, you can download the core program in C++ from from http://apeters.web.cern.ch/apeters/pd2021/raid4.c 

In [18]:
n = 4
src_file = 'raid4.input'
all_bytes = get_arr(src_file)
all_bytes

# We calculate how many zeros have to be added in the end
extra = n - len(all_bytes)%n
bytes_arr = np.hstack([all_bytes,np.zeros(extra, dtype = np.uint8)])

# Reshaping array
nrows = len(bytes_arr)//n
ncols = n
bytes_arr = bytes_arr.reshape((nrows,ncols))
## Rendered as unsigned integers of 8 bits.
bytes_arr

FileNotFoundError: [Errno 2] No such file or directory: 'raid4.input'

In [19]:

# Iterate over the columns and generate the files
for i, v in enumerate(bytes_arr.T):
    post_arr(v, f'raid{n}.{i}')

# Evaluating parity between stripes
row_p = xor_arr(bytes_arr.T)

#Store the row parity
post_arr(row_p,f'raid{n}.{n}')

NameError: name 'bytes_arr' is not defined

In [20]:

## Unsigned 8bit integer representation of the parity stripe
row_p.T

NameError: name 'row_p' is not defined

### 1.2

### 1.3

### 1.4

### 1.5

## 2. Cryptography 

The Caesar cipher is named for Julius Caesar, who
used an alphabet where decrypting would shift three
letters to the left.

A friend has emailed you the following text:
K]amua!trgpy

She told you that her encryption algorithm works
similar to the Caesar cipher:
    
• to each ASCI value of each letter I add a secret key
value.
(note that ASCII values range from 0 to 255)

• additionally to make it more secure I add a variable (so called) nonce
value to each ASCII number.

The nonce start value is 5 for the first character of the message. For each
following character add 1 to the nonce of the previous character, e.g. for
the second letter the nonce added is 6, for the third letter it is 7 aso. 


### 2.1  Is this symmetric or asymmetric encryption and explain why?


This is symmetric encryption because the sender and the receiver only need the public key just containing the information about the shifts of the characters.

### 2.2 Write a small brute force program which tests keys from 0..255 and use a dictionary approach to figure out the original message. In Python you can use the ord() function to get an integer representation of a character and the chr() to retrieve a character string from an integer! 

In [6]:
#!pip install pyenchant
import re, string, enchant
d = enchant.Dict("en_US")

In [13]:
def reshift(letter,ascimin,ascimax): #decrypt method function
    if ord(letter) < ascimin:
        return chr(ord(letter) - ascimin + ascimax + 1)
    elif ord(letter) > ascimax:
        return chr(-ascimax + ord(letter) + ascimin - 1)
    else:
        return letter

def decrypt(mex,nonce0,shift,ascimin,ascimax): #decryption function
    word = []
    nonce = nonce0 #choose nonce0 as the starting value
    
    for letter in mex:
        shiftedchr = chr(ord(letter) - nonce + shift) #nonce must be negative in decrypting
        correctedshiftedchr = reshift(reshift(shiftedchr,ascimin,ascimax),ascimin,ascimax)
        word.append(correctedshiftedchr)
        nonce = nonce + 1 
        
    return word

def stringtowords(phrase): #decrypt setting proposal function    
    clear_word = ''
    for char in phrase:
        if char not in string.punctuation:
            clear_word += char
    return clear_word.lower().split()

In [14]:
ascimax = 255
ascimin = 0
mex = 'K]amua!trgpy'
nonce0 = 5

for i in range(ascimin,ascimax):
    decrypted = decrypt(mex,nonce0,i,ascimin,ascimax)
    words = stringtowords(decrypted)
    for word in words:
        if len(word) != 1:
            try:
                d.check(word)
                if d.check(word) == True:
                    print(i,':',' '.join(words))
            except:
                pass

10 : padova rocks
25 : ps prz
246 : mpbm ow


In [15]:
print("The used key is 10, the original message text is Padova rocks!")

The used key is 10, the original message text is Padova rocks!


## 3. Object Storage Technology

In an object storage system we are mapping objects by name to locations
using a hash table.

Imagine we have a system with ten hard disks (10 locations). We
enumerate the location of a file using an index of the hard disk [0..9]. 

Our hash algorithm for placement produces hashes, which are distributed
uniform over the value space for a flat input key distribution.

We want now to simulate the behaviour of our hash algorithm without
the need to actually compute any hash value.

Instead of using real filenames, which we would hash and map using a
hash table to a location (as we did in the exercise), we are ‘computing’ a
location for ‘any’ file by generating a random number for the location
in the range [0..9] to assign a file location. To place a file in the storage
system we use this random location where the file will be stored and
consumes space. 

Assume each disk has 1TB of space, we have 10TB in total.

Place as many files of 10GB size as possible to hard disks choosing
random locations until one hard disk is full.

Hint: a hard disk is full once you have stored hundred 10GB files.

### 3.1

#### 3.1.a

#### 3.1.b

### 3.2 

#### 3.2.a

#### 3.2.b

### 3.3

### 3.4

## 4. REST APIs and Block Chain Technology